# PyMISP - An interactive tutorial: Basics

## Connecting to MISP
### Your configuration

In [ ]:
# The URL of the MISP instance to connect to
misp_url = 'http://127.0.0.1:8080/'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'BSip0zVadeFDeolkX2g7MHx8mrlr0uE04hh6CQj0'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

# Getting the API key (automatically generated on the trainig VM)

In [ ]:
from pathlib import Path

api_file = Path('apikey')
if api_file.exists():
    misp_url = 'http://127.0.0.1'
    misp_verifycert = False
    with open(api_file) as f:
        misp_key = f.read().strip()
    print(misp_key)

# Initialize PyMISP

In [ ]:
from pymisp import ExpandedPyMISP, PyMISP

misp = ExpandedPyMISP(misp_url, misp_key, misp_verifycert)
misp_old = PyMISP(misp_url, misp_key, misp_verifycert)

## Creating a MISP Event

### Directly

In [ ]:
event = misp.new_event(distribution=1,
               threat_level_id=1,
               analysis=1,
               info="Event from notebook")
print("Event id: %s" % event.id)

In [ ]:
event = misp_old.new_event(distribution=1,
               threat_level_id=1,
               analysis=1,
               info="Event from notebook")
print("Event id: %s" % event['Event']['id'])

### Using the MISPEvent constructor

In [ ]:
from pymisp import MISPEvent

event_obj = MISPEvent()
event_obj.distribution = 1
event_obj.threat_level_id = 1
event_obj.analysis = 1
event_obj.info = "Event from notebook 2"
event = misp.add_event(event_obj)
event_id = event.id
print("Event id: %s" % event_id)

## Fetching a MISP Event

In [ ]:
# Fetch by ID
event = misp.get_event(event_id)
print(event)

In [ ]:
# Fetch by ID
event = misp_old.get_event(event_id)
print(event)

## Adding Attribute to an event

### Adding directly

In [ ]:
attr_type = "ip-src"
value = "8.8.8.8"
category = "Network activity"
to_ids = False
proposal = False
updated_event = misp.add_named_attribute(event,
                                 attr_type,
                                 value,
                                 category=category,
                                 to_ids=to_ids,
                                 proposal=proposal)
print(updated_event)

### Using the MISPAttribute constructor

In [ ]:
from pymisp import MISPAttribute

# Attribute data already defined
attribute = MISPAttribute()
attribute.type = attr_type
attribute.value = value
attribute.category = category
attribute.proposal = proposal
print(attribute)

In [ ]:
# An attribute can also be loaded directly from a JSON
json = '''{
    "type": "ip-dst",
    "value": "127.0.0.1",
    "category": "Network activity",
    "to_ids": false,
    "proposal": false
    }'''

attribute = MISPAttribute()
attribute.from_json(json)
print(attribute)

### And then, update the event

In [ ]:
# Add the attribute to the event
## Fetch the event from MISP
event_dict = misp_old.get(event_id)['Event']

## Convert it to a PyMISP Event
event = MISPEvent()
event.from_dict(**event_dict)

## Add the attribute to the event
event.add_attribute(**attribute)
event.add_attribute(type='domain', value='circl.lu', disable_correlation=True)

## Push the updated event to MISP
event_dict = misp.update_event(event)
print(event_dict)

In [ ]:
# New Python 3.6 API
event = misp.get(event_id)

## Add the attribute to the event
event.add_attribute(**attribute)
event.add_attribute(type='domain', value='circl.lu', disable_correlation=True)

## Push the updated event to MISP
event_dict = misp.update_event(event)
print(event_dict)

## Performing search

### Events by their info fields

In [ ]:
result = misp.search_index(eventinfo='notebook')
events = result['response']

print('Found ', len(events), ' events!')
for event in events:
    print(event['id'], ':', event['info'])

In [ ]:
results[0]

### Attributes by their values

### Search in all attributes

In [ ]:
# Search attributes (specified in controller) where the attribute type is 'ip-src'
# And the to_ids flag is set
attributes = misp.search(controller='attributes', type_attribute='ip-src', to_ids=0, pythonify=True)

# Collect all event_id matching the searched attribute
event_ids = set()
for attr in attributes:
    event_ids.add(attr.event_id)

# Fetch all related events
for event_id in event_ids:
    event = misp.get_event(event_id)
    print(event.info)

## Creating and adding a MISP Object

In [ ]:
from pymisp import MISPObject

object_name = 'email'
object_data = {
    'from': 'admin@admin.test',
    'to': 'admin@foo.bar',
    'subject': 'An email',
}

# Create the MISP Object
misp_obj = MISPObject(object_name)
for obj_relation, value in object_data.items():
    if obj_relation == 'subject':
        misp_obj.add_attribute(obj_relation, value=value, comment='My fancy subject', disable_correlation=True)
    else: 
        misp_obj.add_attribute(obj_relation, value=value)

template_id = misp.get_object_template_id(misp_obj.template_uuid)

# Add the object to MISP
response = misp.add_object(event_id,
                           template_id,
                           misp_obj)
print('Event ID', event_id)
print(response)

# Direct call, no validation

In [ ]:
# The URL of the MISP instance to connect to
#misp_url = 'http://127.0.0.1:8080/'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
#misp_key = 'BSip0zVadeFDeolkX2g7MHx8mrlr0uE04hh6CQj0'
# Should PyMISP verify the MISP certificate
#misp_verifycert = False

from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert)
misp.direct_call('attributes/add/58', {'type': 'ip-dst', 'value': '8.11.8.8'})

In [ ]:
# The URL of the MISP instance to connect to
misp_url = 'http://127.0.0.1:8080/'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'fk5BodCZw8owbscW8pQ4ykMASLeJ4NYhuAbshNjo'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert)
misp.direct_call('attributes/add/2167', '{"type": "ip-dst", "value": "8.8.8.9"}')

In [ ]:
# The URL of the MISP instance to connect to
#misp_url = 'http://127.0.0.1:8080/'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
#misp_key = 'fk5BodCZw8owbscW8pQ4ykMASLeJ4NYhuAbshNjo'
# Should PyMISP verify the MISP certificate
#misp_verifycert = False

from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert)
misp.direct_call('events')